In [1]:
import pandas as pd 
import numpy as np
import networkx as nx 
import json 
import csv 
import matplotlib as plt 
import cdlib
import os


In [67]:
G = list()


for y in range(2012,2021):
   
    g = nx.read_weighted_edgelist(f'../data/years/{str(y)}.weighted.edgelist')
    G.append(g)


maxW = 0
for g in G:
    weights = [g.edges[edge]['weight'] for edge in g.edges()]
    if max(weights) > maxW : maxW = max(weights) 

for g in G:
    for edge in g.edges():
        g.edges[edge]['dissimilarity'] =maxW + 1 - g.edges[edge]['weight']

In [68]:


for g in G:
    weights = [g.edges[edge]['weight'] for edge in g.edges()]
    maxW = max(weights) 
    for edge in g.edges():
        g.edges[edge]['dissimilarity'] =maxW + 1 - g.edges[edge]['weight']

    

In [4]:
for e in list(g.edges)[:10]: print(e)

('0', '1979')
('0', '2238')
('0', '2648')
('0', '8615')
('0', '8841')
('0', '10252')
('0', '11278')
('0', '12564')
('0', '13935')
('0', '14005')


In [3]:
def coreNode(g,alfa,weight):
    
    
    def localCloseNeighborThreshold(g,p,weight):     
        geomin = 1   
        for q in g.neighbors(p): geomin *= g.edges[(p,q)][weight]
        return geomin ** (1 / len(list(g.neighbors(p))))
    
    def localCloseNeighbors(g,p,tresholds,weight):
        return [ q for q in g.neighbors(p) if g.edges[(p,q)][weight] <= tresholds[p] and g.edges[(p,q)][weight] <= tresholds[q] ]
    
        
    
    localCloseNeighborThresholds = {p:localCloseNeighborThreshold(g,p,weight) for p in g.nodes()}
    
    corenodes = list()
    for node in g.nodes:
        
        localMinimumClusteringThreshold = alfa * len(list(g.neighbors(node)))
        treshold = localCloseNeighborThresholds[node]
        
        if len(localCloseNeighbors(g,node,localCloseNeighborThresholds,weight)) >= localMinimumClusteringThreshold: 
            
            corenodes.append(node)
        
    return corenodes

In [33]:
allcores = list()
for g in G:
    cores = coreNode(g,0.2,'dissimilarity')
    allcores.append(cores)
    print(len(g.edges()),' ',len(g.nodes()),' ',len(cores))

19305   3013   639
56023   4949   906
85569   6399   1257
90204   8148   1058
92469   9451   1151
93468   10256   1864
104198   10846   1157
98616   11397   2886
75629   8818   2458


In [9]:
for i, cores in enumerate(allcores): 
    print(len(G[i].edges()),' ',len(G[i].nodes()),' ',len(cores))

19305   3013   97
56023   4949   95
85569   6399   145
90204   8148   46
92469   9451   48
93468   10256   70
104198   10846   57
98616   11397   166
75629   8818   458


In [27]:
def dyCores(G,alfa=1,weight='weight'):
    
    communities = list()
    
    t = 0 
    for g in G:
        print(t)
        t+=1
        core = coreNode(g,alfa,weight)
        for node in g.nodes():
            g.nodes[node]['mark'] = 'unvisited'
            if node in core: g.nodes[node]['core']=True 
            else : g.nodes[node]['core']=False
     
        community = dict()
        k = 0
        
        
        for p in g.nodes():
            
            if g.nodes[p]['mark'] == 'visited' : continue
            if g.nodes[p]['core'] == False :
                g.nodes[p]['mark'] = 'border'
                continue
                
            candidates = set(g.neighbors(p))
            community[k] = list(p)
            
            
            while len(candidates) != 0:
                
                q = candidates.pop()
                
                if g.nodes[q]['core'] == True :
                    
                    if g.nodes[q]['mark'] != 'visited':
                        g.nodes[q]['mark'] = 'visited'
                        community[k].append(q)
                    
                else:
                        
                    for x in g.neighbors(q):
                        if g.nodes[x]['mark'] == 'unvisited' : candidates.add(x)
                        elif  g.nodes[x]['mark'] == 'border' : community[k].append(x) 
                        
                        community[k].append(q)
                        g.nodes[q]['mark'] = 'visited'
                        
            k = k + 1
               
        #OUTLIERS are the last community, are the remaining border nodes
        community[k] = list(node for node in g.nodes() if g.nodes[node]['mark'] == 'border')  
       
                    
        communities.append(community)
    
    return communities
                        
                    
                    
                
            
            
        
        
        
        
        

In [32]:
def dyCores(G,alfa=0.9,weight='weight'):
    
    dynamic_communities = list()
    
    t = 0 
    for g in G:
        print(t)
        
        outliers = set()
        components =  [c for c in sorted(nx.connected_components(g), key=len, reverse=True)]
        for c in components[1:]: 
            for node in c: outliers.add(node) 
        
        g = nx.subgraph(g,components[0])
                      
        
        t+=1
        core = coreNode(g,alfa,weight)
        for node in g.nodes():
            g.nodes[node]['mark'] = 'visited'
            if node in core: g.nodes[node]['core']=True 
            else : g.nodes[node]['core']=False
            
     
        community = dict()
        k = 0
        communities = list()
        
        for p in g.nodes():
            
            if g.nodes[p]['mark'] == 'visited' : continue
            if g.nodes[p]['core'] == False :
                g.nodes[p]['mark'] = 'border'
                continue
                
            candidates = set(g.neighbors(p))
            communities.append(list(p))
            
            
            while len(candidates) != 0:
                
                q = candidates.pop()
                
                if g.nodes[q]['mark'] != 'visited':
                        
                        community[k].append(q)
                        
                        if g.nodes[q]['core'] == True :

                                if g.nodes[q]['mark'] != 'visited':
                                    g.nodes[q]['mark'] = 'visited'
                                    community[k].append(q)
                                    for x in g.neighbors(q):
                                        if g.nodes[x]['mark'] == 'unvisited' : candidates.add(x)
                                            
                        else: g.nodes[q]['mark'] = 'border'
                                  

            k = k + 1
               
        #OUTLIERS are the last community, are the remaining border nodes
        community[k] = list(node for node in g.nodes() if g.nodes[node]['mark'] == 'border')  
       
                    
        communities.append(community)
    
    return communities
                        
                    
                    
                
            
            
        
        
        
        
        

In [12]:
def dyCores(G,alfa=1,weight='weight'):
    
    dynamic_communities = list()
    
    t = 0 
    
    for g in G:
        
        print(t)
        t+=1
        core = coreNode(g,alfa,weight)
        for node in g.nodes():
            g.nodes[node]['mark'] = 'unvisited'
            if node in core: g.nodes[node]['core']=True 
            else : g.nodes[node]['core']=False
     
        communities = dict()
        community = list()
        k = 0
        
        
        for p in g.nodes():
            
            if g.nodes[p]['mark'] == 'visited' or g.nodes[p]['core'] == False: continue
                            
            candidates = set(g.neighbors(p))
            community.append(list(p))
            
            while len(candidates) != 0:
                
                q = candidates.pop()
                
                if g.nodes[q]['mark'] != 'visited':
                    
                    if g.nodes[q]['core'] == True :   
                        community[k].append(q)
                        g.nodes[q]['mark'] = 'visited'
                        for x in g.neighbors(q):
                            if g.nodes[x]['mark'] == 'unvisited' : candidates.add(x)

                    else: g.nodes[q]['mark'] = 'border'
                                  

            k = k + 1
               
        #OUTLIERS are the last community, are the remaining border nodes
        
        
                    
        communities['community'] = community
        communities['outliers'] = list(node for node in g.nodes() if g.nodes[node]['mark'] == 'border')
        communities['coverage']= 1
        communities['algorithm'] = 'dycores'
        
    return communities
                        
                    
                    
                
            
            
        
        
        
        
        

In [112]:
def multicore(G,alfa=0.3,weight='weight',minsize = 4, voteweight = 'centrality'):
    
    #PREPARATION
    g = G.copy()
    core = coreNode(g,alfa,weight)
    communities = list()
    centrality = 1
    k = 0
    borders = set()
    
    for node in g.nodes():
        g.nodes[node]['visited'] = False
        g.nodes[node]['community'] = None
        if node in core: g.nodes[node]['core']=True 
        else : g.nodes[node]['core']=False


            
    #INITIALIZE COMMUNITIES
    for p in g.nodes():

        if g.nodes[p]['visited'] == True or g.nodes[p]['core'] == False: continue
        
        candidates = set(g.neighbors(p))
        
        communities.append([p])
        g.nodes[p]['community'] = k
        g.nodes[p]['centrality'] = centrality
        g.nodes[p]['visited'] = True
        
        while len(candidates) != 0:

            q = candidates.pop()
           
            if g.nodes[q]['visited'] == False:

                if g.nodes[q]['core'] == True :   
                    communities[k].append(q)
                    g.nodes[q]['community'] = k
                    g.nodes[q]['centrality'] = centrality
                    for x in g.neighbors(q):
                        if g.nodes[x]['visited'] == False : candidates.add(x)
                
                else: borders.add(q)
                g.nodes[q]['visited'] = True

        k = k + 1
        
    
    #PROPAGATION
  
    while len(borders) != 0:
        newborders = set() 
        for p in borders: 
            votes = [0 for c in communities]
            for q in g.neighbors(p):
                if q not in borders:
                    if g.nodes[q]['community'] != None:
                        if voteweight == 'centrality': votes[g.nodes[q]['community']] += 1 / g.nodes[q]['centrality']
                        elif voteweight == 'weight': votes[g.nodes[q]['community']] += 1 / g.edges[(p,q)][weight]
                    else: newborders.add(q)
            k = votes.index(max(votes))
            communities[k].append(p)
            g.nodes[p]['community'] = k
            g.nodes[p]['centrality'] = centrality
            
        borders = newborders
        centrality += 1
        

 
       #OUTLIERS 
    unmarked = list()
    for k,c  in enumerate(communities): 
        if len(c) <= minsize :
            
            for p in c: 
                g.nodes[p]['community'] = None
                
            communities[k] = []    
            unmarked.extend(c)
            
    borders = set()
    for p in unmarked:
        for q in g.neighbors(p):
            if g.nodes[q]['community'] != None:
                borders.add(p)
                break
                
    borders = set(borders)
    
   
    while len(borders) != 0:
        newborders = set() 
        for p in borders: 
            votes = [0 for c in communities]
            for q in g.neighbors(p):
                if q not in borders:
                    if g.nodes[q]['community'] != None:
                        if voteweight == 'centrality': votes[g.nodes[q]['community']] += 1 / g.nodes[q]['centrality']
                        elif voteweight == 'weight': votes[g.nodes[q]['community']] += 1 / g.edges[(p,q)][weight]
                    else: newborders.add(q)
            k = votes.index(max(votes))
            communities[k].append(p)
            g.nodes[p]['community'] = k
            g.nodes[p]['centrality'] = 1 + min([g.nodes[q]['centrality'] for q in g.neighbors(p) if g.nodes[q]['community'] == k ])
            
        borders = newborders
        
            

    communities = [c for c in communities if len(c) != 0 ]
    data = dict()
    data['communities'] = communities
    data['outliers'] = list(node for node in g.nodes() if g.nodes[node]['community'] == None)
    data['coverage']= 1
    data['algorithm'] = 'multicore'
    data['centralities'] = {node:g.nodes[node]['centrality'] for node in g.nodes}
        
    return data
                        
                    
                    
                
            
            

In [113]:
comm = multicore(G[3],0.3,'dissimilarity',minsize = 100)

In [71]:
for node in list(G[3].nodes())[:10]:
    print(G[3].nodes[node])

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [114]:
print(len(comm['communities']),[len(c ) for c in comm['communities']])

7 [223, 3795, 2413, 957, 217, 435, 106]


In [60]:
print(np.average([len(c ) for c in comm['communities']]))

44.80107526881721


In [24]:
comms = dyCores(G,0.2,'dissimilarity')

0
1
2


KeyboardInterrupt: 

In [18]:
for comm in comms:
    print([len(c) for c in sorted(comm['communities'][:-1],key=len,reverse=True)])

TypeError: string indices must be integers

In [15]:
for comm in comms:
    print([len(c) for i,c in comm.items()])

AttributeError: 'str' object has no attribute 'items'